## Export Data Script "Blanket Script"


This script searches for ArcGIS Online feature layers and exports them as a geodatabase on ArcGIS Online. This script is based on a system-wide backup script and utilizes a keyword search function to archive data.
To start running the script, you can click Run at the top to run the selected cell. Running a header cell (like this) will just move on to the next cell. A number will appear next to the cell when it has been successfully ran. You can also run them all by clicking "Cell" in the menu bar at the top and then click "Run All". You can run one cell several times if you are tinkering with the code, as long as you make sure to run the first cell below to log yourself into AGOL before you begin (and you run any other cells that create data used in the cell you are tinkering).

In [ ]:
from arcgis.gis import GIS
import datetime as dt
gis = GIS('home')

Starting the script, the cell below will gather the ArcGIS Online layers you want to back up. See look at the comments for specifics. This script will find any feature layer with the keyword you identify in the first open line of this cell. 
For example, if you use "Bolder Wildfire" as your keyword, this script will return everything with "Bolder" OR "Wildfire" in the title. After you run the script, you will get an output message of the layers to be backed up. Make sure you check what the script finds BEFORE setting a task to automatically run this script. This script will only export feature layers. It will find feature layer views, but they will not be exported.
Comments start with "#"

In [ ]:
# What folder is this data going to? Put the folder name between the "". I suggest not using spaces in the folder names. You do not have to create a whole new folder to run the script.
# Leaving this blank will make the data backup to the root folder in your ArcGIS Online.
agol_folder = ""

# How many items do you want to search for and backup at one time? This is the max number of layers that will be backed up each time the script is ran.
# Without this, the default is 10. You can change this number to be whatever you want
num_items = 10

This cell looks for items in AGOL. Put your desired keyword between the "" in the first line. Whatever you put here has to also be in the items' title, otherwise the code won't know to look for them.

In [ ]:
keyword = "<INCIDENT NAME>"
query_string = "type:Feature Service, title:{}, owner:{}".format(keyword, username)
items = gis.content.search(query=query_string, max_items=num_items, sort_field='modifed', sort_order='desc')
print(str(len(items)) + " items will be backed up to " + agol_folder +  " Folder. See the list below:")
items

This section is what does the magic. By default, you do not have to change anything here. However, if you want to change the name of the output file, this would be done in line 10 next to "result". Do not worry if several lines say "An error occurred downloading". This just means that that file is not a feature layer and isn't going to download. If everything gives you this error, check your code and make sure you are grabbing feature layers.

In [ ]:
def download_as_fgdb(item_list):
    for item in item_list:
        try:
            if 'View Service' in item.typeKeywords:
                print(item.title + " is view, not downloading")
            else: 
                print("Downloading " + item.title)
                # This line figures out today's date in Year-Month-Day format, then puts the date at the end of the export data name
                version=time.strftime('%Y%m%d %H:%M')
                result = item.export(version + " UTC " + item.title, "File Geodatabase")
                print("Successfully downloaded " + item.title)
                result.move(folder= agol_folder)
        except Exception as e:
            print("An error occurred downloading " + item.title)
            print(e)
    print("The function has completed")

download_as_fgdb(items)